In [2]:
import pandas as pd
import numpy as np
import os

In [37]:
log = "log_"
threads = "threads.csv"

samp_coord = ["1k-1k", "10k-1k", "1k-10k"]
sparsities = ["0005", "0010", "0200", "1000"]
hw = "hogwild"
es = "exampleshared"
ei = "exampleindependent"
sh = "segmentedhogwild"

subfolders = [f.path for f in os.scandir("F:\data") if f.is_dir() ]
print(subfolders[30])

df = pd.read_csv(subfolders[30] + "\\" + log + str(1) + threads)
times = df.iloc[:,0].values
df.iloc[:, 1] = df.iloc[:, 1].astype(str).str.extract('(\d+)').astype(float)
df.iloc[:, -1] = df.iloc[:, -1].astype(str).str.extract('(\d+)').astype(float)
iterates = df.iloc[:, 1:].values
print(iterates.shape)

F:\data\hogwild_linearregression_1k-10k-0200-r
(100, 10000)


C:\Users\Sriram\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()
C:\Users\Sriram\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
